In [1]:
from __future__ import print_function
%matplotlib inline
import os
import numpy as np
import pandas as pd
from time import time
from gurobipy import *
from random import *
import matplotlib.pyplot as plt
from random import randint,choice,random
plt.style.use("ggplot")
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider
import ipywidgets as widgets
from HRH import *
instances = 'instances/'
from collections import namedtuple
import itertools

Item = namedtuple("Item", ['id', 'size'])
Candidate = namedtuple("Candidate", ['items', 'fitness'])

In [2]:
@interact
def AG(instance = os.listdir(instances), schema = ['S2','S1'], greedy = ['FF','BF'], method=['TS','RW','RS','SUS'], 
            details = ['Non','Oui'],
            Population = IntSlider( min=0, max=100, step=1,value=0, continuous_update=False,layout = dict(width='80%')),
            generations = IntSlider( min=0, max=150, step=1,value=100, continuous_update=False,layout = dict(width='80%')),
            TS_proba = IntSlider( min=0, max=100, step=1,value=70, continuous_update=False,layout = dict(width='80%')),
            crossover_proba = IntSlider( min=0, max=100, step=1,value=40, continuous_update=False,layout = dict(width='80%')),
            mutation_proba = IntSlider( min=0, max=100, step=1,value=40, continuous_update=False,layout = dict(width='80%')),
           N_tabu = IntSlider( min=1, max=17, step=1,value=8, continuous_update=False,layout = dict(width='80%')),
           Nb_iter = IntSlider( min=0, max=7000, step=50,value=3500, continuous_update=False,layout = dict(width='80%')),
           Temperature = IntSlider( min=0, max=30000, step=100,value=21000, continuous_update=False,layout = dict(width='80%'))
      ):
    print('S1 : AG -> Recherche tabou')
    print('S2 : AG -> Recuit simule')
    df = pd.read_csv('datasets.csv',index_col=0)
    opt = df.loc[instance.split('.')[0]]['value']
    f = open(instances+instance,'r')
    n = int(f.readline())
    c = int(f.readline())
    Wj = list(map (lambda l : int(l.replace("\n",'')), f.readlines()))
    f.close()
    print('n : ',n,' | c : ',c,' | opt : ',opt)
    if (details == 'Oui'):
        detail = True
    else:
        detail = False
    sol = []
    if(Population > 0 and generations > 0 and Nb_iter >0 and schema == 'S1'):
        sol = hybrid_AG_TS(n, Wj, c, Population, generations, 2, TS_proba/100.0, 
                crossover_proba/100.0, mutation_proba/100.0, greedy, False, method, 
                           N_tabu, Nb_iter, Init_FF=True)
        show_sol_tabu(c,sol,opt,detail)
        print(len(sol[0]))
    elif(Population > 0 and generations > 0 and Nb_iter >0 and schema == 'S2'):
        sol = hybrid_AG_SA(n, Wj, c, Population, generations, 2, TS_proba/100.0, crossover_proba/100.0, 
                        mutation_proba/100.0, greedy, False, method, Temperature, Nb_iter)
        show_sol_SA(c,sol,opt,detail)

interactive(children=(Dropdown(description='instance', options=('HARD0.txt', 'HARD7.txt', 'N1C2W4_P.txt', 'N1C…